<a href="https://colab.research.google.com/github/KIMDOKYOUNG/DeepLearning/blob/master/chapter5/chapter_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 밑바닥부터 시작하는 딥러닝 1 뽀개기
#### CHAPTER 5 오차역전파법 
#### 일자 : 2020-08-16

- 수치미분은 구현이 쉽지만 계산 시간이 오래걸린다. 
- 가중치 매개변수의 기울기를 효율적으로 계산하는 오차역전파법!

---

## 덧셈, 곱셈의 역전파 => 계산 그래프를 사용해서 계산 과정을 보면 쉽다. 

- 덧셈의 역전파에서는 상류의 값을 그대로 흘려보내서 순방향 입력 신호의 값이 필요하지 않다.
- 곱섬의 역전파는 상류의 값에 순방향 입력 신호를 서로 바꿔서 곱한 값을 흘려보낸다.  

## 곱셈 계층 


In [1]:
class MulLayer:
  
  def __init(self):
    self.x = none
    self.y = none
  
  def forward(self,x,y):
    self.x = x
    self.y = y
    out = x * y

    return out
  
  def backward(self, d_out): # 상류의 값(미분 값)에 순방향 입력 신호를 서로 바꿔서 곱한 값을 흘려보낸다. 
    d_x = d_out * self.y
    d_y = d_out * self.x

    return d_x, d_y 


## 곱셈 계층 순전파 

In [2]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(apple_price)
print(price)

200
220.00000000000003


## 곱셈 계층 역전파 

In [3]:
dprice = 1

dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


## 덧셈 계층

In [4]:
class AddLayer:

  def __init__(self):
    pass

  def forward(self,x,y):
    out = x + y
    return out

  def backward(self, d_out):
    d_x = d_out * 1
    d_y = d_out * 1
    return d_x, d_y

## 사과 2개 귤 3개 사는 상황 구현 

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

dprice = 1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
mul_tax_layer = MulLayer()
add_apple_orange_layer = AddLayer()

apple_price = mul_apple_layer.forward(apple,apple_num)
orange_price = mul_orange_layer.forward(orange,orange_num)
fruit_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(fruit_price, tax)

print("**** 순전파 ****\n")
print("apple_price : {} \n orange_price : {} \n fruit_price : {} \n price : {}\n".format(apple_price, orange_price, fruit_price, price))


dfruit_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dfruit_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
print("**** 역전파 ****\n")
print("dapple : {} \n dapple_num : {} \n dapple_price : {} \n dorange : {} \n dorange_num : {} \n dorange_price : {} \n dfruit_price : {} \n dprice : {}".format(dapple, dapple_num ,dapple_price, dorange, dorange_num, dorange_price, dfruit_price, dprice))



**** 순전파 ****

apple_price : 200 
 orange_price : 450 
 fruit_price : 650 
 price : 715.0000000000001

**** 역전파 ****

dapple : 2.2 
 dapple_num : 110.00000000000001 
 dapple_price : 1.1 
 dorange : 3.3000000000000003 
 dorange_num : 165.0 
 dorange_price : 1.1 
 dfruit_price : 1.1 
 dprice : 1


## 활성화 함수 계층 구현하기 

### 1. ReLU 계층 

In [6]:
class ReLU:
  def __init__(self):
    self.x = none
  
  def forward(self, x):
    self.x = x
    
    return self.x
  
  def backward(self, d_out):
    if self.x > 0:
      return 1
    else :
      return 0

내가 생각했던 코드... 하지만 어딘가 이상하다 

masking을 해야 된다! 

In [8]:
import  numpy as np 

x = np.array([[1.0,-0.5],[-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)


[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [10]:
x[mask] = 0

In [11]:
x

array([[1., 0.],
       [0., 3.]])

True가 0이 된다. 즉, 0보다 작은 수가 0이 된다.

In [12]:
class ReLU:
  def __init__(self):
    self.mask = none
  
  def forward(self, x):
    self.mask = (x <= 0)
    out = x.copy()
    out[maxk] = 0

    return out
  
  def backward(self, d_out):
    d_out[self.mask] = 0
    d_x = d_out

    return d_x